In [1]:
import os
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
base_model = MobileNet(weights='imagenet', include_top=False) 

/home/marcolini/anaconda3/envs/deep_myq/lib/python3.6/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [4]:
DATASET_DIR = '../myq_dataset/Selfie-dataset/labelled/'

In [43]:
images = os.listdir(DATASET_DIR)

images = pd.DataFrame(images)
images = images.rename(index=int, columns={0: "filename"})

In [46]:
images.head()

,filename,negative,neutral,positive,male,female,baby,child,teenager,youth,middle_age,senior
0,2_1_3_925473_240060952784357_1812030612_a.jpg,0,0,1,0,1,0,0,0,1,0,0
1,2_1_3_1171037_236837476520319_348230056_a.jpg,0,0,1,0,1,0,0,0,1,0,0
2,2_1_3_10261202_614305691993395_506992923_a.jpg,0,0,1,0,1,0,0,0,1,0,0
3,1_1_3_10013334_555648927867451_503825726_a.jpg,0,1,0,0,1,0,0,0,1,0,0
4,1_1_3_10261211_754796697894552_907528579_a.jpg,0,1,0,0,1,0,0,0,1,0,0


In [45]:
images['negative'] = images['filename'].apply(lambda x: (int(x.split('_')[0] == '0')))
images['neutral'] = images['filename'].apply(lambda x: (int(x.split('_')[0] == '1')))
images['positive'] = images['filename'].apply(lambda x: (int(x.split('_')[0] == '2')))

images['male'] = images['filename'].apply(lambda x: (int(x.split('_')[1] == '0')))
images['female'] = images['filename'].apply(lambda x: (int(x.split('_')[1] == '1')))

images['baby'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '0')))
images['child'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '1')))
images['teenager'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '2')))
images['youth'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '3')))
images['middle_age'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '4')))
images['senior'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '5')))

In [49]:
images.sum()

filename      2_1_3_925473_240060952784357_1812030612_a.jpg2...
negative                                                   3124
neutral                                                   11136
positive                                                   7900
male                                                       6001
female                                                    16159
baby                                                         53
child                                                       395
teenager                                                   2678
youth                                                     18237
middle_age                                                  787
senior                                                       10
dtype: object

In [56]:
X_train, X_test, y_train, y_test = train_test_split(images['filename'], images[images.columns[1:]]) 

In [63]:
images.columns[1:]

Index(['negative', 'neutral', 'positive', 'male', 'female', 'baby', 'child',
       'teenager', 'youth', 'middle_age', 'senior'],
      dtype='object')

In [86]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [87]:
columns = images.columns[1:].tolist()

In [88]:
train_generator = train_datagen.flow_from_dataframe(dataframe=images[:16000], 
                                                    directory=DATASET_DIR, 
                                                    x_col='filename', 
                                                    y_col=columns,
                                                    class_mode='other',
                                                    target_size=(224,224),
                                                    color_mode='rgb')

test_generator = test_datagen.flow_from_dataframe(dataframe=images[16000:], 
                                                    directory=DATASET_DIR, 
                                                    x_col='filename', 
                                                    y_col=columns,
                                                    class_mode=None,
                                                    target_size=(224,224),
                                                    color_mode='rgb')

Found 16000 images.
Found 6163 images.


In [85]:
next(train_generator)[1]

array([[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0,

In [91]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) 
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x) 

output_layers = []
for i in range(11):
    output_layers.append(Dense(1, activation = 'sigmoid')(x))

model = Model(inputs=base_model.input, outputs=output_layers)

In [92]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 3 864         conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 3 128         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

In [94]:
model.compile(optimizer='Adam',loss=['binary_crossentropy']*11,metrics=['accuracy'])

In [112]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x, [batch_y[:,i] for i in range(11)])

In [113]:
next(generator_wrapper(train_generator))[1]

AttributeError: 'list' object has no attribute 'shape'

In [114]:
step_size_train = train_generator.n//train_generator.batch_size
step_size_test = test_generator.n//test_generator.batch_size

In [115]:
model.fit_generator(generator=generator_wrapper(train_generator),
                    steps_per_epoch=step_size_train)

Epoch 1/1
 53/500 [==>...........................] - ETA: 15:33 - loss: 3.7943 - dense_18_loss: 0.4296 - dense_19_loss: 0.7316 - dense_20_loss: 0.6483 - dense_21_loss: 0.3509 - dense_22_loss: 0.3830 - dense_23_loss: 0.0338 - dense_24_loss: 0.1135 - dense_25_loss: 0.3951 - dense_26_loss: 0.4920 - dense_27_loss: 0.1944 - dense_28_loss: 0.0221 - dense_18_acc: 0.8520 - dense_19_acc: 0.5607 - dense_20_acc: 0.6356 - dense_21_acc: 0.8361 - dense_22_acc: 0.8225 - dense_23_acc: 0.9811 - dense_24_acc: 0.9764 - dense_25_acc: 0.8803 - dense_26_acc: 0.8113 - dense_27_acc: 0.9528 - dense_28_acc: 0.9894

KeyboardInterrupt: 

In [93]:

#model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=2)

#model.load_weights('./model.h5')

def load_image(img_path, show=False):
    
    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]
    
    if show:
        plt.imshow(img_tensor[0])
        plt.axis('off')
        plt.show()
    
    return img_tensor

#img_path = 'C:/Users/Ferhat/Python Code/Workshop/Tensoorflow transfer learning/blue_tit.jpg'
img_path = './test.jpg'
new_image = load_image(img_path)

pred = model.predict(new_image)
print(pred)

model.save_weights("./model.h5")

OSError: Unable to open file (unable to open file: name = './model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
for layer in model.layers[:87]:
    layer.trainable=False
for layer in model.layers[87:]:
    layer.trainable=True

In [ ]:
model = MobileNet(weights='imagenet', include_top=False)

In [ ]:
model.summary()

In [ ]:
DATASET_DIR = '../myq_dataset/Selfie-dataset/labelled/'

In [ ]:
images = os.listdir(DATASET_DIR)

In [ ]:
im = Image.open(os.path.join(DATASET_DIR, images[0]))

In [ ]:
plt.imshow(im)

In [ ]:
img = image.load_img(os.path.join(DATASET_DIR, images[0]), target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

In [ ]:
print('Predicted:', decode_predictions(preds, top=5)[0])